In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

# ---------- Load data ----------
dtype = {
    "store_nbr": "category",
    "family": "category",
    "sales": "float32",
    "onpromotion": "float32", # Changed from uint64 to float32 to handle NA values
}
df = pd.read_csv("train.csv", dtype=dtype, parse_dates=["date"])
df["date"] = pd.to_datetime(df["date"]) # Explicitly convert to datetime
df["date"] = df["date"].dt.to_period("D")

# pick one series (edit as you like)
FAMILY = "SCHOOL AND OFFICE SUPPLIES"
y_df = (
    df[df["family"] == FAMILY]
    .groupby("date")[["sales", "onpromotion"]]
    .mean()
    .sort_index()
)
y = y_df["sales"]
promo = (y_df["onpromotion"] > 0).astype(int)

# ---------- Baseline features: trend + annual Fourier ----------
fourier = CalendarFourier(freq="A", order=3)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,            # trend
    additional_terms=[fourier],
    drop=True,
)
X = dp.in_sample()

# ---------- Two-stage model (baseline -> residual uplift) ----------
def two_stage_predict(X_train, promo_train, y_train, X_test, promo_test):
    # Stage 1: baseline fitted on non-promo rows
    base = sm.OLS(y_train[promo_train == 0], X_train[promo_train == 0]).fit()
    base_pred = base.predict(X_test)

    # Stage 2: residual model using promo-derived features (simple version from your code)
    oof_base = base.predict(X_train)
    residual = y_train - oof_base

    promo_all = pd.concat([promo_train, promo_test])
    Z = pd.DataFrame({"promo": promo_all})
    Z["roll_mean_7"] = promo_all.shift(1).rolling(7, min_periods=1).mean()
    Z["roll_std_7"]  = promo_all.shift(1).rolling(7, min_periods=2).std(ddof=0)

    valid = residual.dropna().index.intersection(Z.dropna().index)
    uplift = sm.OLS(residual.loc[valid], Z.loc[valid]).fit()
    uplift_pred = uplift.predict(Z.loc[promo_test.index])

    return base_pred + uplift_pred

# ---------- One-stage baseline (for comparison) ----------
def one_stage_cv_rmse(X, y, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    rmses = []
    for tr, va in tscv.split(X):
        tr_idx, va_idx = y.index[tr], y.index[va]
        m = sm.OLS(y.loc[tr_idx], X.loc[tr_idx]).fit()
        pred = m.predict(X.loc[va_idx])
        rmses.append(np.sqrt(mean_squared_error(y.loc[va_idx], pred)))
    return float(np.mean(rmses))

def two_stage_cv_rmse(X, promo, y, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    rmses = []
    for tr, va in tscv.split(X):
        tr_idx, va_idx = y.index[tr], y.index[va]
        pred = two_stage_predict(X.loc[tr_idx], promo.loc[tr_idx], y.loc[tr_idx],
                                 X.loc[va_idx], promo.loc[va_idx])
        rmses.append(np.sqrt(mean_squared_error(y.loc[va_idx], pred)))
    return float(np.mean(rmses))

if __name__ == "__main__":
    rmse_1 = one_stage_cv_rmse(X, y)
    rmse_2 = two_stage_cv_rmse(X, promo, y)
    print(f"[{FAMILY}] One-stage baseline CV RMSE: {rmse_1:.4f}")
    print(f"[{FAMILY}] Two-stage (baseline+uplift) CV RMSE: {rmse_2:.4f}")
    print(f"Relative improvement: {(rmse_1 - rmse_2) / rmse_1 * 100:.2f}%")

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)


[SCHOOL AND OFFICE SUPPLIES] One-stage baseline CV RMSE: 6.4456
[SCHOOL AND OFFICE SUPPLIES] Two-stage (baseline+uplift) CV RMSE: 6.4497
Relative improvement: -0.06%


In [7]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

# ---------------------------
# 1) Data prep: build daily y and promo for a given family
# ---------------------------
def load_family_daily(train_csv_path: str, family: str, agg="sum"):
    df = pd.read_csv(train_csv_path, parse_dates=["date"])
    dff = df[df["family"] == family].copy()

    if agg == "sum":
        g = dff.groupby("date")[["sales", "onpromotion"]].sum()
    elif agg == "mean":
        g = dff.groupby("date")[["sales", "onpromotion"]].mean()
    else:
        raise ValueError("agg must be 'sum' or 'mean'")

    g = g.asfreq("D").fillna(0.0)
    # keep DatetimeIndex (more stable with statsmodels)
    y = g["sales"].astype(float)
    promo = g["onpromotion"].astype(float)
    return y, promo

# ---------------------------
# 2) Calendar features: trend + annual Fourier + DOW
# ---------------------------
def build_calendar_X(index, K=3, add_dow=True):
    # statsmodels: use YE (A deprecated)
    fourier = CalendarFourier(freq="YE", order=K)
    dp = DeterministicProcess(
        index=index,
        constant=True,
        order=1,  # linear trend
        additional_terms=[fourier],
        drop=True
    )
    X = dp.in_sample()

    if add_dow:
        dow = index.dayofweek
        dow_df = pd.get_dummies(dow, prefix="dow", drop_first=True).astype(float)
        dow_df.index = index
        X = X.join(dow_df)

    # ensure numeric
    X = X.apply(pd.to_numeric, errors="coerce").astype(float)
    return X

# ---------------------------
# 3) Walk-forward backtest (rolling window): 4y train -> 15d horizon
#    Baseline-only vs Two-stage (baseline + uplift)
# ---------------------------
def walk_forward_rmse_baseline_only(
    y, promo,
    K=3,
    train_days=365*4,
    horizon=15,
    step=7,
    fit_on_nonpromo=True,
):
    df = pd.concat([y.rename("y"), promo.rename("promo")], axis=1).dropna()
    y = df["y"]; promo = df["promo"]
    idx = y.index
    Xcal_all = build_calendar_X(idx, K=K, add_dow=True)

    rmses = []
    for t0 in range(train_days, len(idx) - horizon + 1, step):
        tr = idx[t0-train_days:t0]
        va = idx[t0:t0+horizon]

        y_tr, y_va = y.loc[tr], y.loc[va]
        p_tr = promo.loc[tr]

        X_tr = Xcal_all.loc[tr].copy()
        X_va = Xcal_all.loc[va].copy()

        if fit_on_nonpromo:
            mask = (p_tr == 0)
            X_fit = X_tr.loc[mask]
            y_fit = y_tr.loc[mask]
            # if too few non-promo points, fall back to all
            if len(X_fit) < 30:
                X_fit, y_fit = X_tr, y_tr
        else:
            X_fit, y_fit = X_tr, y_tr

        m1 = sm.OLS(y_fit, X_fit).fit()
        y_hat = pd.Series(m1.predict(X_va), index=va)

        rmse = np.sqrt(mean_squared_error(y_va, y_hat))
        rmses.append(rmse)

    return float(np.mean(rmses)), rmses

def walk_forward_rmse_two_stage(
    y, promo,
    K=3,
    train_days=365*4,
    horizon=15,
    step=7,
    fit_stage1_on_nonpromo=True,
    roll_window=7,
    gate_uplift_to_promo=False,   # set True if you want uplift=0 when promo==0
):
    df = pd.concat([y.rename("y"), promo.rename("promo")], axis=1).dropna()
    y = df["y"]; promo = df["promo"]
    idx = y.index
    Xcal_all = build_calendar_X(idx, K=K, add_dow=True)

    rmses = []
    for t0 in range(train_days, len(idx) - horizon + 1, step):
        tr = idx[t0-train_days:t0]
        va = idx[t0:t0+horizon]

        y_tr, y_va = y.loc[tr], y.loc[va]
        p_tr, p_va = promo.loc[tr], promo.loc[va]

        # ---- Stage 1 baseline ----
        X1_tr = Xcal_all.loc[tr].copy()
        X1_va = Xcal_all.loc[va].copy()

        if fit_stage1_on_nonpromo:
            mask = (p_tr == 0)
            X_fit = X1_tr.loc[mask]
            y_fit = y_tr.loc[mask]
            if len(X_fit) < 30:
                X_fit, y_fit = X1_tr, y_tr
        else:
            X_fit, y_fit = X1_tr, y_tr

        m1 = sm.OLS(y_fit, X_fit).fit()
        base_tr = pd.Series(m1.predict(X1_tr), index=tr)
        base_va = pd.Series(m1.predict(X1_va), index=va)

        resid_tr = y_tr - base_tr

        # ---- Stage 2 uplift on residual ----
        # build promo stats on train+val together (no leakage: rolling uses shift(1))
        p_tv = pd.concat([p_tr, p_va])
        Z_tv = pd.DataFrame(index=p_tv.index)
        Z_tv["promo"] = p_tv
        Z_tv[f"promo_rm{roll_window}"] = p_tv.shift(1).rolling(roll_window, min_periods=1).mean()
        Z_tv[f"promo_rs{roll_window}"] = p_tv.shift(1).rolling(roll_window, min_periods=1).std()

        Z_tr = Z_tv.loc[tr].fillna(0.0)
        Z_va = Z_tv.loc[va].fillna(0.0)

        Z_tr_c = sm.add_constant(Z_tr, has_constant="add")
        Z_va_c = sm.add_constant(Z_va, has_constant="add")

        m2 = sm.OLS(resid_tr, Z_tr_c).fit()
        uplift_va = pd.Series(m2.predict(Z_va_c), index=va)

        if gate_uplift_to_promo:
            uplift_va = uplift_va.where(p_va > 0, 0.0)

        y_hat = base_va + uplift_va

        rmse = np.sqrt(mean_squared_error(y_va, y_hat))
        rmses.append(rmse)

    return float(np.mean(rmses)), rmses

# ---------------------------
# 4) Run and print the exact "resume-style" improvement
# ---------------------------
if __name__ == "__main__":
    TRAIN_PATH = "train.csv"
    FAMILY = "SCHOOL AND OFFICE SUPPLIES"  # or "CLEANING"

    # use SUM to match the "large RMSE" scale
    y, promo = load_family_daily(TRAIN_PATH, FAMILY, agg="sum")

    # Baseline vs Two-stage walk-forward CV
    K = 3
    train_days = 365*4
    horizon = 15
    step = 7

    rmse1, _ = walk_forward_rmse_baseline_only(
        y, promo, K=K, train_days=train_days, horizon=horizon, step=step, fit_on_nonpromo=True
    )
    rmse2, _ = walk_forward_rmse_two_stage(
        y, promo, K=K, train_days=train_days, horizon=horizon, step=step,
        fit_stage1_on_nonpromo=True, roll_window=7, gate_uplift_to_promo=False
    )

    rel = (rmse1 - rmse2) / rmse1 * 100.0

    print(f"[{FAMILY}] One-stage baseline CV RMSE: {rmse1:.4f}")
    print(f"[{FAMILY}] Two-stage (baseline+uplift) CV RMSE: {rmse2:.4f}")
    print(f"Relative improvement: {rel:.2f}%")

[SCHOOL AND OFFICE SUPPLIES] One-stage baseline CV RMSE: 401.5464
[SCHOOL AND OFFICE SUPPLIES] Two-stage (baseline+uplift) CV RMSE: 242.1043
Relative improvement: 39.71%


In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jan 20 16:32:14 2026

@author: chensha
"""

import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

def build_calendar_X(index, K=3, add_dow=True):
    """Stage1: trend + annual Fourier + optional DOW"""
    # Changed 'A' to 'YE' to address FutureWarning
    fourier = CalendarFourier(freq="YE", order=K)
    dp = DeterministicProcess(
        index=index,
        constant=True,
        order=1,                 # linear trend
        additional_terms=[fourier],
        drop=True
    )
    X = dp.in_sample()

    if add_dow:
        # index should be PeriodIndex('D') or DatetimeIndex
        dt_index = X.index.to_timestamp() if isinstance(X.index, pd.PeriodIndex) else X.index
        dow = dt_index.dayofweek
        X = X.join(pd.get_dummies(dow, prefix="dow", drop_first=True).set_index(X.index))

    # Ensure all columns are numeric after joining DOW dummies
    X = X.apply(pd.to_numeric, errors="coerce").astype(float)
    return X

def make_lags(series, lags, name_prefix):
    Z = pd.DataFrame(index=series.index)
    for L in lags:
        Z[f"{name_prefix}_lag{L}"] = series.shift(L)
    return Z

def rolling_cv_two_stage(
    y, promo,
    K=3,
    train_days=365*4,
    horizon=15,
    step=7,
    stage1_y_lags=(),          # for Config A
    stage2_promo_lags=(1,7,14),
    stage2_resid_lags=(),      # for Config B
    use_stage2=True,
    only_fit_stage1_on_nonpromo=True
):
    """
    y, promo: pd.Series indexed by daily date (PeriodIndex('D') recommended).
    """
    assert y.index.equals(promo.index)
    idx = y.index

    X_cal_all = build_calendar_X(idx, K=K, add_dow=True)

    all_rmse = []
    promo_rmse = []

    for t0 in range(train_days, len(idx) - horizon + 1, step):
        tr = idx[t0-train_days:t0]
        va = idx[t0:t0+horizon]

        y_tr, y_va = y.loc[tr], y.loc[va]
        p_tr, p_va = promo.loc[tr], promo.loc[va]

        # ---- Stage 1 features (calendar + optional y lags) ----
        X1_tr = X_cal_all.loc[tr].copy()
        X1_va = X_cal_all.loc[va].copy()

        if stage1_y_lags:
            # build y lags using train+val concatenation to avoid leakage
            y_tv = pd.concat([y_tr, y_va])
            L_tv = make_lags(y_tv, stage1_y_lags, "y")
            X1_tr = X1_tr.join(L_tv.loc[tr])
            X1_va = X1_va.join(L_tv.loc[va])

        # align (drop NaNs caused by lags)
        X1_tr = X1_tr.loc[X1_tr.dropna().index]
        y_tr_aligned = y_tr.loc[X1_tr.index]
        p_tr_aligned = p_tr.loc[X1_tr.index]

        # fit stage1 on non-promo days if desired
        if only_fit_stage1_on_nonpromo:
            mask = (p_tr_aligned == 0)
            X_fit = X1_tr.loc[mask]
            y_fit = y_tr_aligned.loc[mask]
        else:
            X_fit = X1_tr
            y_fit = y_tr_aligned

        m1 = sm.OLS(y_fit, X_fit).fit()

        base_tr = pd.Series(m1.predict(X1_tr), index=X1_tr.index)
        base_va = pd.Series(m1.predict(X1_va), index=X1_va.index)

        # ---- Stage 2 (uplift/residual model) ----
        if use_stage2:
            resid_tr = (y_tr_aligned.loc[base_tr.index] - base_tr)

            # build stage2 features using train+val concatenation
            p_tv = pd.concat([p_tr, p_va])

            Z_tv = pd.DataFrame(index=p_tv.index)

            # promo lags
            if stage2_promo_lags:
                Z_tv = Z_tv.join(make_lags(p_tv, stage2_promo_lags, "promo"))

            # promo rolling stats (shift(1) to avoid using current day promo if you want strict causality)
            Z_tv["promo_rm7"]  = p_tv.shift(1).rolling(7,  min_periods=1).mean()
            Z_tv["promo_rs7"]  = p_tv.shift(1).rolling(7,  min_periods=1).std()
            Z_tv["promo_rm14"] = p_tv.shift(1).rolling(14, min_periods=1).mean()
            Z_tv["promo_rs14"] = p_tv.shift(1).rolling(14, min_periods=1).std()

            # optional residual lags (Config B typical)
            if stage2_resid_lags:
                # need residuals for train part; extend with NaNs for val then lag works
                resid_tv = pd.concat([resid_tr.reindex(tr), pd.Series(index=va, dtype=float)])
                Z_tv = Z_tv.join(make_lags(resid_tv, stage2_resid_lags, "resid"))

            Z_tr = Z_tv.loc[tr].dropna()
            # align residual
            resid_fit = resid_tr.reindex(Z_tr.index).dropna()
            Z_tr = Z_tr.loc[resid_fit.index]

            if len(Z_tr) == 0:
                uplift_va = pd.Series(0.0, index=va)
            else:
                m2 = sm.OLS(resid_fit, sm.add_constant(Z_tr)).fit()
                Z_va = Z_tv.loc[va].fillna(0.0)
                uplift_va = pd.Series(
                    m2.predict(sm.add_constant(Z_va, has_constant="add")),
                    index=va
                )
        else:
            uplift_va = pd.Series(0.0, index=va)

        # Address FutureWarning for fillna(method=) by using ffill() and direct fillna(0.0)
        pred_va = base_va.reindex(va).ffill() + uplift_va.reindex(va).fillna(0.0)

        # ---- metrics ----
        rmse_all = np.sqrt(mean_squared_error(y_va, pred_va))
        all_rmse.append(rmse_all)

        promo_mask = (p_va > 0)
        if promo_mask.any():
            rmse_p = np.sqrt(mean_squared_error(y_va[promo_mask], pred_va[promo_mask]))
            promo_rmse.append(rmse_p)

    return {
        "rmse_all_mean": float(np.mean(all_rmse)),
        "rmse_promo_mean": float(np.mean(promo_rmse)) if promo_rmse else np.nan,
        "n_folds": len(all_rmse),
        "n_promo_folds": len(promo_rmse)
    }

# ------------------ Usage example ------------------
# 1) load and aggregate one family to daily mean (match your untitled16 style)
train = pd.read_csv("train.csv", parse_dates=["date"])
df = train[train["family"] == "SCHOOL AND OFFICE SUPPLIES"].copy()

g = df.groupby("date")[["sales","onpromotion"]].mean()
g = g.asfreq("D").fillna(0.0)
g.index = g.index.to_period("D")

y = g["sales"]
promo = g["onpromotion"]

# Config A: lag in Stage1
res_A = rolling_cv_two_stage(
    y, promo,
    K=3, train_days=365*4, horizon=15, step=7,
    stage1_y_lags=(1,7),        # <-- lag@Stage1
    stage2_promo_lags=(1,7,14),
    stage2_resid_lags=(),
    use_stage2=True
)

# Config B: lag in Stage2 (residual lags)
res_B = rolling_cv_two_stage(
    y, promo,
    K=3, train_days=365*4, horizon=15, step=7,
    stage1_y_lags=(),           # <-- no y-lag in Stage1
    stage2_promo_lags=(1,7,14),
    stage2_resid_lags=(1,7),    # <-- lag@Stage2 on residual
    use_stage2=True
)

print("Config A (lag@Stage1):", res_A)
print("Config B (lag@Stage2):", res_B)


Config A (lag@Stage1): {'rmse_all_mean': 3.1311253670082575, 'rmse_promo_mean': 4.011587591313507, 'n_folds': 31, 'n_promo_folds': 25}
Config B (lag@Stage2): {'rmse_all_mean': 6.110221384609565, 'rmse_promo_mean': 7.892502217762042, 'n_folds': 31, 'n_promo_folds': 25}


In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jan 20 17:00:15 2026

@author: chensha
"""

import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def build_calendar_X(index, K=3, add_dow=True):
    fourier = CalendarFourier(freq="YE", order=K)
    dp = DeterministicProcess(index=index, constant=True, order=1,
                             additional_terms=[fourier], drop=True)
    X = dp.in_sample()

    if add_dow:
        dow = index.dayofweek
        dow_df = pd.get_dummies(dow, prefix="dow", drop_first=True).astype(float)
        dow_df.index = index
        X = X.join(dow_df)

    return X.apply(pd.to_numeric, errors="coerce").astype(float)

def walk_forward_cv_two_stage(
    y, promo,
    K=3,
    train_days=365*4,
    horizon=15,
    step=7,
    fit_stage1_on_nonpromo=True,
    roll_window=7,
    gate_uplift_to_promo=False,
):
    df = pd.concat([y.rename("y"), promo.rename("promo")], axis=1).dropna()
    y = df["y"].astype(float)
    promo = df["promo"].astype(float)
    idx = y.index

    Xcal_all = build_calendar_X(idx, K=K, add_dow=True)

    fold_rmses = []
    for t0 in range(train_days, len(idx) - horizon + 1, step):
        tr = idx[t0-train_days:t0]
        va = idx[t0:t0+horizon]

        y_tr, y_va = y.loc[tr], y.loc[va]
        p_tr, p_va = promo.loc[tr], promo.loc[va]

        # ---- Stage 1 baseline ----
        X1_tr = Xcal_all.loc[tr].copy()
        X1_va = Xcal_all.loc[va].copy()

        if fit_stage1_on_nonpromo:
            mask = (p_tr == 0)
            X_fit = X1_tr.loc[mask]
            y_fit = y_tr.loc[mask]
            if len(X_fit) < 30:  # fallback if too few non-promo points
                X_fit, y_fit = X1_tr, y_tr
        else:
            X_fit, y_fit = X1_tr, y_tr

        # FORCE numeric to avoid object dtype issues
        X_fit = X_fit.apply(pd.to_numeric, errors="coerce").astype(float)
        y_fit = pd.to_numeric(y_fit, errors="coerce").astype(float)
        ok = X_fit.notna().all(axis=1) & y_fit.notna()
        X_fit, y_fit = X_fit.loc[ok], y_fit.loc[ok]

        m1 = sm.OLS(y_fit, X_fit).fit()
        base_tr = pd.Series(m1.predict(X1_tr), index=tr)
        base_va = pd.Series(m1.predict(X1_va), index=va)

        resid_tr = y_tr - base_tr

        # ---- Stage 2 uplift features ----
        p_tv = pd.concat([p_tr, p_va])
        Z_tv = pd.DataFrame(index=p_tv.index)
        Z_tv["promo"] = p_tv
        Z_tv[f"promo_rm{roll_window}"] = p_tv.shift(1).rolling(roll_window, min_periods=1).mean()
        Z_tv[f"promo_rs{roll_window}"] = p_tv.shift(1).rolling(roll_window, min_periods=1).std()

        Z_tr = Z_tv.loc[tr].fillna(0.0)
        Z_va = Z_tv.loc[va].fillna(0.0)

        Z_tr_c = sm.add_constant(Z_tr, has_constant="add")
        Z_va_c = sm.add_constant(Z_va, has_constant="add")

        m2 = sm.OLS(resid_tr, Z_tr_c).fit()
        uplift_va = pd.Series(m2.predict(Z_va_c), index=va)

        if gate_uplift_to_promo:
            uplift_va = uplift_va.where(p_va > 0, 0.0)

        y_hat = base_va + uplift_va
        fold_rmses.append(rmse(y_va, y_hat))

    return float(np.mean(fold_rmses)), fold_rmses
def select_params_by_cv(
    y, promo,
    train_days=365*4,
    horizon=15,
    step=7,
    K_grid=(2, 3, 4, 6),
    roll_grid=(3, 7, 14),
    fit_stage1_nonpromo_grid=(True, False),
    gate_grid=(False, True),
):
    rows = []
    for K in K_grid:
        for rw in roll_grid:
            for nonpromo in fit_stage1_nonpromo_grid:
                for gate in gate_grid:
                    mean_rmse, fold_rmses = walk_forward_cv_two_stage(
                        y, promo,
                        K=K,
                        train_days=train_days,
                        horizon=horizon,
                        step=step,
                        fit_stage1_on_nonpromo=nonpromo,
                        roll_window=rw,
                        gate_uplift_to_promo=gate,
                    )
                    rows.append({
                        "K": K,
                        "roll_window": rw,
                        "stage1_nonpromo": nonpromo,
                        "gate_uplift": gate,
                        "rmse_cv_mean": mean_rmse,
                        "n_folds": len(fold_rmses),
                    })
    df = pd.DataFrame(rows).sort_values("rmse_cv_mean").reset_index(drop=True)
    return df
train = pd.read_csv("train.csv", parse_dates=["date"])
# Corrected family name from 'sCH' to 'SCHOOL AND OFFICE SUPPLIES'
df = train[train["family"] == "SCHOOL AND OFFICE SUPPLIES"].copy()

g = df.groupby("date")[["sales","onpromotion"]].mean()
g = g.asfreq("D").fillna(0.0)
g.index = g.index.to_period("D")

y = g["sales"]
promo = g["onpromotion"]

df = select_params_by_cv(y, promo, train_days=365*4, horizon=15, step=7)
print(df.head(10))
best = df.iloc[0]
print("BEST:", best.to_dict())

   K  roll_window  stage1_nonpromo  gate_uplift  rmse_cv_mean  n_folds
0  6           14             True         True      4.303823       31
1  2           14             True         True      4.308289       31
2  3           14             True         True      4.334794       31
3  4           14             True         True      4.342428       31
4  2           14             True        False      4.380945       31
5  6           14             True        False      4.387348       31
6  2            7             True         True      4.397450       31
7  6            7             True         True      4.402718       31
8  3           14             True        False      4.408484       31
9  4           14             True        False      4.415627       31
BEST: {'K': 6, 'roll_window': 14, 'stage1_nonpromo': True, 'gate_uplift': True, 'rmse_cv_mean': 4.303823037423984, 'n_folds': 31}


In [16]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

def rmse(y_true, y_pred) -> float:
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def build_calendar_X(index, K=3, add_dow=True):
    # freq="A" deprecated -> use "YE"
    fourier = CalendarFourier(freq="YE", order=K)
    dp = DeterministicProcess(
        index=index,
        constant=True,
        order=1,  # linear trend
        additional_terms=[fourier],
        drop=True
    )
    X = dp.in_sample()

    if add_dow:
        dow = index.dayofweek
        dow_df = pd.get_dummies(dow, prefix="dow", drop_first=True).astype(float)
        dow_df.index = index
        X = X.join(dow_df)

    # ensure numeric
    return X.apply(pd.to_numeric, errors="coerce").astype(float)

def add_stage1_y_lags(X_base: pd.DataFrame, y: pd.Series, lags=(1, 7, 14)):
    """
    Add y_lagk columns to Stage1 design matrix.
    Uses y.shift(k), so for day t we only use y up to t-k.
    """
    X = X_base.copy()
    for k in lags:
        X[f"y_lag{k}"] = y.shift(k)
    return X

def walk_forward_cv_baseline_only(
    y, promo,
    K=3,
    train_days=365*4,
    horizon=15,
    step=7,
    fit_on_nonpromo=True,
):
    df = pd.concat([y.rename("y"), promo.rename("promo")], axis=1).dropna()
    y = df["y"].astype(float)
    promo = df["promo"].astype(float)
    idx = y.index

    Xcal_all = build_calendar_X(idx, K=K, add_dow=True)

    fold_rmses = []
    for t0 in range(train_days, len(idx) - horizon + 1, step):
        tr = idx[t0-train_days:t0]
        va = idx[t0:t0+horizon]

        y_tr, y_va = y.loc[tr], y.loc[va]
        p_tr = promo.loc[tr]

        X_tr = Xcal_all.loc[tr]
        X_va = Xcal_all.loc[va]

        if fit_on_nonpromo:
            mask = (p_tr == 0)
            X_fit = X_tr.loc[mask]
            y_fit = y_tr.loc[mask]
            if len(X_fit) < 30:  # fallback
                X_fit, y_fit = X_tr, y_tr
        else:
            X_fit, y_fit = X_tr, y_tr

        # force numeric + drop NaNs
        X_fit = X_fit.apply(pd.to_numeric, errors="coerce").astype(float)
        y_fit = pd.to_numeric(y_fit, errors="coerce").astype(float)
        ok = X_fit.notna().all(axis=1) & y_fit.notna()
        X_fit, y_fit = X_fit.loc[ok], y_fit.loc[ok]

        m1 = sm.OLS(y_fit, X_fit).fit()
        y_hat = pd.Series(m1.predict(X_va), index=va)

        # Ensure y_hat has no NaNs before RMSE calculation
        y_hat = y_hat.ffill().fillna(0.0)

        fold_rmses.append(rmse(y_va, y_hat))

    return float(np.mean(fold_rmses)), fold_rmses

def walk_forward_cv_two_stage_A_lag_stage1(
    y, promo,
    K=3,
    stage1_y_lags=(1, 7, 14),      # <-- A版本核心：lag 放 Stage1
    train_days=365*4,
    horizon=15,
    step=7,
    fit_stage1_on_nonpromo=True,
    roll_window=7,
    gate_uplift_to_promo=False,    # optional: set True to force uplift=0 when promo=0
):
    df = pd.concat([y.rename("y"), promo.rename("promo")], axis=1).dropna()
    y = df["y"].astype(float)
    promo = df["promo"].astype(float)
    idx = y.index

    Xcal_all = build_calendar_X(idx, K=K, add_dow=True)

    fold_rmses = []
    for t0 in range(train_days, len(idx) - horizon + 1, step):
        tr = idx[t0-train_days:t0]
        va = idx[t0:t0+horizon]

        y_tr, y_va = y.loc[tr], y.loc[va]
        p_tr, p_va = promo.loc[tr], promo.loc[va]

        # ---- Stage 1 baseline with y-lags ----
        X1_tr_base = Xcal_all.loc[tr]
        X1_va_base = Xcal_all.loc[va]

        # IMPORTANT: build lag features using the FULL y (shifted), then slice
        X1_all = add_stage1_y_lags(Xcal_all, y, lags=stage1_y_lags)
        X1_tr = X1_all.loc[tr]
        X1_va = X1_all.loc[va]

        # Stage1 train subset (often non-promo)
        if fit_stage1_on_nonpromo:
            mask = (p_tr == 0)
            X_fit = X1_tr.loc[mask]
            y_fit = y_tr.loc[mask]
            if len(X_fit) < 30:
                X_fit, y_fit = X1_tr, y_tr
        else:
            X_fit, y_fit = X1_tr, y_tr

        # force numeric + drop NaNs (lags create NaNs early in window)
        X_fit = X_fit.apply(pd.to_numeric, errors="coerce").astype(float)
        y_fit = pd.to_numeric(y_fit, errors="coerce").astype(float)
        ok = X_fit.notna().all(axis=1) & y_fit.notna()
        X_fit, y_fit = X_fit.loc[ok], y_fit.loc[ok]

        m1 = sm.OLS(y_fit, X_fit).fit()
        base_tr = pd.Series(m1.predict(X1_tr), index=tr)
        base_va = pd.Series(m1.predict(X1_va), index=va)

        resid_tr = y_tr - base_tr

        # ---- Stage 2 uplift on residual (promo stats only) ----
        p_tv = pd.concat([p_tr, p_va])
        Z_tv = pd.DataFrame(index=p_tv.index)
        Z_tv["promo"] = p_tv
        Z_tv[f"promo_rm{roll_window}"] = p_tv.shift(1).rolling(roll_window, min_periods=1).mean()
        Z_tv[f"promo_rs{roll_window}"] = p_tv.shift(1).rolling(roll_window, min_periods=1).std()

        Z_tr = Z_tv.loc[tr].fillna(0.0)
        Z_va = Z_tv.loc[va].fillna(0.0)

        Z_tr_c = sm.add_constant(Z_tr, has_constant="add")
        Z_va_c = sm.add_constant(Z_va, has_constant="add")

        m2 = sm.OLS(resid_tr, Z_tr_c).fit()
        uplift_va = pd.Series(m2.predict(Z_va_c), index=va)

        if gate_uplift_to_promo:
            uplift_va = uplift_va.where(p_va > 0, 0.0)

        # Ensure base_va and uplift_va are clean and aligned before summing
        base_va = base_va.reindex(va).ffill().fillna(0.0)
        uplift_va = uplift_va.reindex(va).fillna(0.0)

        y_hat = base_va + uplift_va
        fold_rmses.append(rmse(y_va, y_hat))

    return float(np.mean(fold_rmses)), fold_rmses
FAMILY = "SCHOOL AND OFFICE SUPPLIES"

# y, promo 需要你自己先聚合好（建议 sum）
# y, promo = load_family_daily("train.csv", FAMILY, agg="sum")  # 你若已有就跳过

K = 3
train_days = 365*4
horizon = 15
step = 7
train = pd.read_csv("train.csv", parse_dates=["date"])
df = train[train["family"] == "SCHOOL AND OFFICE SUPPLIES"].copy()

g = df.groupby("date")[["sales","onpromotion"]].mean()
g = g.asfreq("D").fillna(0.0)
g.index = g.index.to_period("D")

y = g["sales"]
promo = g["onpromotion"]

rmse_base, _ = walk_forward_cv_baseline_only(
    y, promo, K=K, train_days=train_days, horizon=horizon, step=step, fit_on_nonpromo=True
)

rmse_A, _ = walk_forward_cv_two_stage_A_lag_stage1(
    y, promo, K=K, stage1_y_lags=(1,7,14),
    train_days=train_days, horizon=horizon, step=step,
    fit_stage1_on_nonpromo=True, roll_window=7,
    gate_uplift_to_promo=False
)

rel = (rmse_base - rmse_A) / rmse_base * 100
print(f"[{FAMILY}] Baseline-only CV RMSE: {rmse_base:.4f}")
print(f"[{FAMILY}] Two-stage A (lag@Stage1) CV RMSE: {rmse_A:.4f}")
print(f"Relative improvement: {rel:.2f}%")

[SCHOOL AND OFFICE SUPPLIES] Baseline-only CV RMSE: 7.4360
[SCHOOL AND OFFICE SUPPLIES] Two-stage A (lag@Stage1) CV RMSE: 2.8759
Relative improvement: 61.32%


In [18]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jan 20 17:54:28 2026

@author: chensha
"""

import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

def rmse(y_true, y_pred) -> float:
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def build_calendar_X(index, K=3, add_dow=True):
    fourier = CalendarFourier(freq="YE", order=K)
    dp = DeterministicProcess(
        index=index,
        constant=True,
        order=1,
        additional_terms=[fourier],
        drop=True
    )
    X = dp.in_sample()

    if add_dow:
        dow = index.dayofweek
        dow_df = pd.get_dummies(dow, prefix="dow", drop_first=True).astype(float)
        dow_df.index = index
        X = X.join(dow_df)

    return X.apply(pd.to_numeric, errors="coerce").astype(float)
def add_y_lags(X, y, lags=(1,7,14)):
    X2 = X.copy()
    for k in lags:
        X2[f"y_lag{k}"] = y.shift(k)
    return X2

def cv_two_stage_A(
    y, promo,
    K=3,
    y_lags=(1,7,14),
    roll_window=7,
    train_days=365*4,
    horizon=15,
    step=15,
    fit_stage1_on_nonpromo=True,
    gate_uplift_to_promo=False
):
    df = pd.concat([y.rename("y"), promo.rename("promo")], axis=1).dropna()
    y = df["y"].astype(float)
    promo = df["promo"].astype(float)
    idx = y.index

    Xcal = build_calendar_X(idx, K=K, add_dow=True)
    X1_all = add_y_lags(Xcal, y, lags=y_lags)

    fold_rmses = []
    for t0 in range(train_days, len(idx)-horizon+1, step):
        tr = idx[t0-train_days:t0]
        va = idx[t0:t0+horizon]

        y_tr, y_va = y.loc[tr], y.loc[va]
        p_tr, p_va = promo.loc[tr], promo.loc[va]

        X1_tr = X1_all.loc[tr]
        X1_va = X1_all.loc[va]

        # Stage1 fit set
        if fit_stage1_on_nonpromo:
            mask = (p_tr == 0)
            X_fit = X1_tr.loc[mask]
            y_fit = y_tr.loc[mask]
            if len(X_fit) < 30:
                X_fit, y_fit = X1_tr, y_tr
        else:
            X_fit, y_fit = X1_tr, y_tr

        X_fit = X_fit.apply(pd.to_numeric, errors="coerce").astype(float)
        y_fit = pd.to_numeric(y_fit, errors="coerce").astype(float)
        ok = X_fit.notna().all(axis=1) & y_fit.notna()
        X_fit, y_fit = X_fit.loc[ok], y_fit.loc[ok]

        m1 = sm.OLS(y_fit, X_fit).fit()
        base_tr = pd.Series(m1.predict(X1_tr), index=tr)
        base_va = pd.Series(m1.predict(X1_va), index=va)

        resid_tr = y_tr - base_tr

        # Stage2 promo stats only
        p_tv = pd.concat([p_tr, p_va])
        Z_tv = pd.DataFrame(index=p_tv.index)
        Z_tv["promo"] = p_tv
        Z_tv[f"promo_rm{roll_window}"] = p_tv.shift(1).rolling(roll_window, min_periods=1).mean()
        Z_tv[f"promo_rs{roll_window}"] = p_tv.shift(1).rolling(roll_window, min_periods=1).std()

        Z_tr = Z_tv.loc[tr].fillna(0.0)
        Z_va = Z_tv.loc[va].fillna(0.0)

        Z_tr_c = sm.add_constant(Z_tr, has_constant="add")
        Z_va_c = sm.add_constant(Z_va, has_constant="add")

        m2 = sm.OLS(resid_tr, Z_tr_c).fit()
        uplift_va = pd.Series(m2.predict(Z_va_c), index=va)

        if gate_uplift_to_promo:
            uplift_va = uplift_va.where(p_va > 0, 0.0)

        # Ensure base_va and uplift_va are clean and aligned before summing
        base_va = base_va.reindex(va).ffill().fillna(0.0)
        uplift_va = uplift_va.reindex(va).fillna(0.0)

        y_hat = base_va + uplift_va
        fold_rmses.append(rmse(y_va, y_hat))

    return float(np.mean(fold_rmses)), fold_rmses

def add_resid_lags(Z, resid, lags=(1,7,14)):
    Z2 = Z.copy()
    for k in lags:
        Z2[f"resid_lag{k}"] = resid.shift(k)
    return Z2

def cv_two_stage_B(
    y, promo,
    K=3,
    resid_lags=(1,7,14),
    roll_window=7,
    train_days=365*4,
    horizon=15,
    step=15,
    fit_stage1_on_nonpromo=True,
    gate_uplift_to_promo=False
):
    df = pd.concat([y.rename("y"), promo.rename("promo")], axis=1).dropna()
    y = df["y"].astype(float)
    promo = df["promo"].astype(float)
    idx = y.index

    Xcal = build_calendar_X(idx, K=K, add_dow=True)

    fold_rmses = []
    for t0 in range(train_days, len(idx)-horizon+1, step):
        tr = idx[t0-train_days:t0]
        va = idx[t0:t0+horizon]

        y_tr, y_va = y.loc[tr], y.loc[va]
        p_tr, p_va = promo.loc[tr], promo.loc[va]

        X1_tr = Xcal.loc[tr]
        X1_va = Xcal.loc[va]

        # Stage1 baseline
        if fit_stage1_on_nonpromo:
            mask = (p_tr == 0)
            X_fit = X1_tr.loc[mask]
            y_fit = y_tr.loc[mask]
            if len(X_fit) < 30:
                X_fit, y_fit = X1_tr, y_tr
        else:
            X_fit, y_fit = X1_tr, y_tr

        X_fit = X_fit.apply(pd.to_numeric, errors="coerce").astype(float)
        y_fit = pd.to_numeric(y_fit, errors="coerce").astype(float)
        ok = X_fit.notna().all(axis=1) & y_fit.notna()
        X_fit, y_fit = X_fit.loc[ok], y_fit.loc[ok]

        m1 = sm.OLS(y_fit, X_fit).fit()
        base_tr = pd.Series(m1.predict(X1_tr), index=tr)
        base_va = pd.Series(m1.predict(X1_va), index=va)

        resid_tr = y_tr - base_tr

        # Stage2 features: promo stats + residual lags (trained on TRAIN ONLY)
        p_tv = pd.concat([p_tr, p_va])
        Z_tv = pd.DataFrame(index=p_tv.index)
        Z_tv["promo"] = p_tv
        Z_tv[f"promo_rm{roll_window}"] = p_tv.shift(1).rolling(roll_window, min_periods=1).mean()
        Z_tv[f"promo_rs{roll_window}"] = p_tv.shift(1).rolling(roll_window, min_periods=1).std()

        # build Z on full timeline, but resid lags must come from TRAIN residuals only (no y leakage)
        resid_full = pd.concat([resid_tr, pd.Series(index=va, dtype=float)])
        Z_tv = add_resid_lags(Z_tv, resid_full, lags=resid_lags)

        Z_tr = Z_tv.loc[tr].fillna(0.0)
        Z_va = Z_tv.loc[va].fillna(0.0)

        Z_tr_c = sm.add_constant(Z_tr, has_constant="add")
        Z_va_c = sm.add_constant(Z_va, has_constant="add")

        m2 = sm.OLS(resid_tr, Z_tr_c).fit()
        uplift_va = pd.Series(m2.predict(Z_va_c), index=va)

        if gate_uplift_to_promo:
            uplift_va = uplift_va.where(p_va > 0, 0.0)

        # Ensure base_va and uplift_va are clean and aligned before summing
        base_va = base_va.reindex(va).ffill().fillna(0.0)
        uplift_va = uplift_va.reindex(va).fillna(0.0)

        y_hat = base_va + uplift_va
        fold_rmses.append(rmse(y_va, y_hat))

    return float(np.mean(fold_rmses)), fold_rmses

def compare_A_B_basic(
    y, promo,
    K_basic=3,
    y_lags_basic=(1,7,14),
    resid_lags_basic=(1,7,14),
    roll_window_basic=7,
    train_days=365*4,
    horizon=15,
    step=15,  # 第一轮建议 15
    fit_stage1_on_nonpromo=True,
    gate_uplift_to_promo=False
):
    rmse_A, _ = cv_two_stage_A(
        y, promo, K=K_basic, y_lags=y_lags_basic, roll_window=roll_window_basic,
        train_days=train_days, horizon=horizon, step=step,
        fit_stage1_on_nonpromo=fit_stage1_on_nonpromo,
        gate_uplift_to_promo=gate_uplift_to_promo
    )
    rmse_B, _ = cv_two_stage_B(
        y, promo, K=K_basic, resid_lags=resid_lags_basic, roll_window=roll_window_basic,
        train_days=train_days, horizon=horizon, step=step,
        fit_stage1_on_nonpromo=fit_stage1_on_nonpromo,
        gate_uplift_to_promo=gate_uplift_to_promo
    )

    winner = "A (lag@Stage1)" if rmse_A < rmse_B else "B (lag@Stage2)"
    rel = (rmse_B - rmse_A) / rmse_B * 100.0  # positive means A better than B
    print(f"Basic settings: K={K_basic}, roll_window={roll_window_basic}, step={step}, nonpromo={fit_stage1_on_nonpromo}, gate={gate_uplift_to_promo}")
    print(f"A (lag@Stage1) RMSE: {rmse_A:.4f}")
    print(f"B (lag@Stage2) RMSE: {rmse_B:.4f}")
    print(f"Winner: {winner}; A vs B relative advantage: {rel:.2f}%")
    return rmse_A, rmse_B
train = pd.read_csv("train.csv", parse_dates=["date"])
df = train[train["family"] == "SCHOOL AND OFFICE SUPPLIES"].copy()

g = df.groupby("date")[["sales","onpromotion"]].mean()
g = g.asfreq("D").fillna(0.0)
g.index = g.index.to_period("D")

y = g["sales"]
promo = g["onpromotion"]

compare_A_B_basic(
    y, promo,
    K_basic=3,
    y_lags_basic=(1,7,14),
    resid_lags_basic=(1,7,14),
    roll_window_basic=7,
    step=15,  # 第一轮快
    fit_stage1_on_nonpromo=True,
    gate_uplift_to_promo=False
)

Basic settings: K=3, roll_window=7, step=15, nonpromo=True, gate=False
A (lag@Stage1) RMSE: 3.0003
B (lag@Stage2) RMSE: 6.4066
Winner: A (lag@Stage1); A vs B relative advantage: 53.17%


(3.0003086442791917, 6.40657200131748)

Here are the specific code changes made in cell `0HDyqjlHrH0q` to address the `NaN` error:

```python
def walk_forward_cv_baseline_only(
    y, promo,
    K=3,
    train_days=365*4,
    horizon=15,
    step=7,
    fit_on_nonpromo=True,
):
    # ... (previous code)

    m1 = sm.OLS(y_fit, X_fit).fit()
    y_hat = pd.Series(m1.predict(X_va), index=va)

    # Ensure y_hat has no NaNs before RMSE calculation
    y_hat = y_hat.ffill().fillna(0.0) # <--- Added this line

    fold_rmses.append(rmse(y_va, y_hat))

    # ... (rest of the function)

def walk_forward_cv_two_stage_A_lag_stage1(
    y, promo,
    K=3,
    stage1_y_lags=(1, 7, 14),
    train_days=365*4,
    horizon=15,
    step=7,
    fit_stage1_on_nonpromo=True,
    roll_window=7,
    gate_uplift_to_promo=False,
):
    # ... (previous code)

    # Ensure base_va and uplift_va are clean and aligned before summing
    base_va = base_va.reindex(va).ffill().fillna(0.0) # <--- Added this line
    uplift_va = uplift_va.reindex(va).fillna(0.0)     # <--- Added this line

    y_hat = base_va + uplift_va
    fold_rmses.append(rmse(y_va, y_hat))

    # ... (rest of the function)
```